# **Apply Text Splitting Techniques to Enhance Model Responsiveness**

In many data processing tasks, especially those involving large documents, breaking down text into smaller, more manageable chunks is essential. Text splitters are tools specifically designed to accomplish this, ensuring that lengthy texts are divided into coherent segments. This division is crucial for maintaining the integrity and readability of the information, making it easier to handle and process. Effective text splitting helps prevent overwhelming systems with large, unwieldy blocks of text and ensures that each segment remains meaningful and contextually relevant.

The significance of text splitters becomes even more apparent in the context of retrieval-augmented generation (RAG). RAG involves fetching relevant pieces of information from a large dataset and using them to generate accurate and context-aware responses. Without properly split text, the retrieval process can become inefficient, potentially missing critical pieces of information or returning irrelevant data. By using text splitters to create well-defined chunks, the retrieval process can be streamlined, ensuring that the most relevant information is easily accessible. This not only enhances the efficiency of data retrieval but also improves the quality and relevance of the generated responses, making text splitters an important tool in the RAG workflow.

<figure>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/3Y4eB0v7LXU5cSVbeN1b8A/text-splitter.png" width="50%" alt="langchain">
    <figcaption><a>source: DALL-E</a></figcaption>
</figure>

*   [`langchain`, `langchain-text-splitters`](https://www.langchain.com/) for using relevant features and text splitters from Langchain.
*   [`lxml`](https://pypi.org/project/lxml/) for libxml2 and libxslt libraries, which is used for splitting html text.


In [ ]:
%%capture
!pip install "langchain==0.2.7"
!pip install "langchain-core==0.2.20"
!pip install "langchain-text-splitters==0.2.2"
!pip install "lxml==5.2.2"

hello


## Text splitters

### Key parameters


When using the splitter, you can customize several key parameters to fit your needs:
- **separator**: Define the characters that will be used for splitting the text.
- **chunk_size**: Specify the maximum size of your chunks to ensure they are as granular or broad as needed.
- **chunk_overlap**: Maintain context between chunks by setting the `chunk_overlap` parameter, which determines the number of characters that overlap between consecutive chunks. This helps ensure that information isn't lost at the chunk boundaries.
- **length_function**: Define how the length of chunks is calculated.

### Prepare the document

A long document has been prepared for this project to demonstrate the performance of each splitter. Run the following code to download it.


In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/YRYau14UJyh0DdiLDdzFcA/companypolicies.txt"

--2025-05-27 20:31:37--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/YRYau14UJyh0DdiLDdzFcA/companypolicies.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15660 (15K) [text/plain]
Saving to: ‘companypolicies.txt’

companypolicies.txt 100%[===================>]  15.29K  --.-KB/s    in 0s      

2025-05-27 20:31:37 (150 MB/s) - ‘companypolicies.txt’ saved [15660/15660]



Let's take a look at what the document looks like.


In [3]:
# This is a long document you can split up.
with open("companypolicies.txt") as f:
    companypolicies = f.read()
print(companypolicies)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

It is a long document about a company's policies.

### Document object

Before introducing the splitters, let's take a look at the document object in LangChain, which is a data structure used to represent and manage text data in RAG process.

A Document object in LangChain contains information about some data. It has two attributes:

- `page_content: str`: The content of this document. Currently is only a string.
- `metadata: dict`: Arbitrary metadata associated with this document. Can track the document id, file name, etc.


In [4]:
from langchain_core.documents import Document
Document(page_content="""Python is an interpreted high-level general-purpose programming language.
                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.""",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "About Python",
             'my_document_create_time' : 1680013019
         })

Document(metadata={'my_document_id': 234234, 'my_document_source': 'About Python', 'my_document_create_time': 1680013019}, page_content="Python is an interpreted high-level general-purpose programming language. \n                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.")

### Split by Character

This is the simplest method, which splits the text based on characters (by default `"\n\n"`) and measures chunk length by the number of characters.
- **How the text is split**: By single character.
- **How the chunk size is measured**: By number of characters.



In the following code, you will use `CharacterTextSplitter` to split the document by character.
- Separator: Set to `''`, meaning that any character can act as a separator once the chunk size reaches the set limit.
- Chunk size: Set to `200`, meaning that once a chunk reaches 200 characters, it will be split.
- Chunk overlap: Set to `20`, meaning there will be `20` characters overlapping between chunks.
- Length function: Set to `len`.


In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

You will use `split_text` function to operate the split.


In [6]:
texts = text_splitter.split_text(companypolicies)

Let's take a look how the document has been split.


In [7]:
texts

['1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built',
 'kplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whe',
 'ur interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value e',
 "iversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrat",
 'erences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws 

After the split, you'll see that the document has been divided into multiple chunks, with some character overlaps between the chunks.

You can see how many chunks you get.


In [8]:
len(texts)

87

 got `87` chunks.


You can also use the following code to add metadata to the text, forming it into a `document` object using LangChain.


In [10]:
companypolicies

"1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violati

In [9]:
texts = text_splitter.create_documents([companypolicies], metadatas=[{"document":"Company Policies"}])  # pass the metadata as well

In [12]:
texts[1]

Document(metadata={'document': 'Company Policies'}, page_content='kplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whe')

### Recursively Split by Character

This text splitter is the recommended one for generic text. It is parameterized by a list of characters, and it tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`.

It processes the large text by attempting to split it by the first character, `\n\n`. If the first split by \n\n results in chunks that are still too large, it moves to the next character, `\n`, and attempts to split by it. This process continues through the list of characters until the chunks are less than the specified chunk size.

This method aims to keep all paragraphs (then sentences, then words) together as much as possible, as these are generally the most semantically related pieces of text.

- **How the text is split**: by list of characters.
- **How the chunk size is measured**: by number of characters.

The `RecursiveCharacterTextSplitter` class from LangChain is used to implement it.
- You use the default separator list, which is `["\n\n", "\n", " ", ""]`.
- Chunk size is set to `100`.
- Chunk overlap is set to `20`.
- And the length function is `len`.


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
texts = text_splitter.create_documents([companypolicies])
texts

[Document(metadata={}, page_content='1.\tCode of Conduct'),
 Document(metadata={}, page_content='Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'),
 Document(metadata={}, page_content='that guide every member of our organization. We are committed to maintaining a workplace that is'),
 Document(metadata={}, page_content='a workplace that is built on integrity, respect, and accountability.'),
 Document(metadata={}, page_content='Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and'),
 Document(metadata={}, page_content='acting honestly and transparently in all our interactions, whether with colleagues, clients, or the'),
 Document(metadata={}, page_content='clients, or the broader community. We respect and protect sensitive information, and we avoid'),
 Document(metadata={}, page_content='and we avoid conflicts of interest.'),
 Document(metadata={}, page_content="Respect: We embrace diversity and valu

In [14]:
len(texts)

215

215 chunks